<a href="https://colab.research.google.com/github/HikariZasureiya/HLV25FinanceScript/blob/main/HLV_finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade gspread
import datetime
# last executed
print("last executed" , datetime.datetime.now())


last executed 2025-04-07 09:00:21.562848


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [70]:
def check_pay_status(sheetno: int, nameworksheet, sheetname: str):
    values = nameworksheet.get_all_values()

    headers = [h.strip().upper() for h in values[0]]
    df = pd.DataFrame(values[1:], columns=headers)


    count = 0
    for status in df['PAID']:
        if str(status).strip().upper() == 'Y':
            count += 1

    print(f'Dept: {sheetname}  Paid: {count} / {len(df)}')

In [40]:
def findit(rolltostream :dict ,sheetno :int , nameworksheet ,sheetname :str):
    values = nameworksheet.get_all_values()
    df = pd.DataFrame(values[1:], columns=values[0])
    tylis = pd.DataFrame(df)
    count = 0
    try:
      for idx, roll in enumerate(tylis['ROLL_NUMBER']):
        try:
          roll = str(roll).strip()
          if pd.notna(roll) and roll in rolltostream:
            count +=1
            print(f"Sheet {sheetno+1} ➜", idx, roll)
        except Exception as e:
          print(e)
          print(f'inner exception')
    except Exception as e:
        print(e)
        print(f'error in sheet {sheetno} , {sheetname}')
    # print(f'Dept: {sheetname}  Paid: {count} / {len(df)}')

In [ ]:
def writeit(rolltostream :dict ,sheetno :int , nameworksheet ,sheetname :str):
    values = nameworksheet.get_all_values()
    df = pd.DataFrame(values[1:], columns=values[0])
    tylis = pd.DataFrame(df)


    try:
      for idx, roll in enumerate(tylis['ROLL_NUMBER']):
        roll = str(roll).strip()
        if pd.notna(roll) and roll in rolltostream:
          row_num = idx+2
          nameworksheet.update_cell(row_num, 3, 'Y')
          nameworksheet.update_cell(row_num, 4, rolltostream[roll][1])

    except Exception as e:
        print(e)
        print(f'error in sheet {sheetno} , {sheetname}')
        # wb.close()

In [42]:
import pandas as pd
from openpyxl import load_workbook

#opens hlv'25 collection spreadsheet
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1s-E2MnVyXGZSobcnaZ4SqJba0Tao4B2Zdoux6-vu4ZU/edit?gid=1018150263#gid=1018150263")
worksheet = sheet.worksheets()[0]
data = worksheet.get_all_records()
fin = pd.DataFrame(data)
rolltostream ={}
for idx , roll in enumerate(fin['University Roll No']):
    temp = ''
    if(fin.loc[idx , 'Food Preference'] == 'Non Veg'):
        temp = 'NON-VEG'
    elif(fin.loc[idx , 'Food Preference'] == 'Veg'):
        temp = 'VEG'
    rolltostream[str(roll)] = [str(fin.loc[idx , 'Stream']) , temp]

print(rolltostream , len(rolltostream))

{'13000123103': ['CSE', 'NON-VEG'], '13000123101': ['CSE', 'NON-VEG'], '13000123117': ['CSE', 'NON-VEG'], '13000124121': ['CSE', 'NON-VEG'], '13000123124': ['CSE', 'NON-VEG'], '13000123095': ['CSE', 'NON-VEG'], '13000123067': ['CSE', 'NON-VEG'], '13000123113': ['CSE', 'NON-VEG'], '13031124051': ['CSBS', 'NON-VEG'], '13000123110': ['CSE', 'VEG'], '13000123090': ['CSE', 'NON-VEG'], '13000123105': ['CSE', 'NON-VEG'], '13000124159': ['CSE', 'VEG'], '13000123018': ['CSE', 'VEG'], '13000123052': ['CSE', 'NON-VEG'], '13000122039': ['CSE', 'NON-VEG']} 16


In [73]:
namelist = gc.open_by_url("https://docs.google.com/spreadsheets/d/1We74wo0hkN73b2Wa3hqtgoqnNAGFtPjls4_NiwkRGwc/edit?gid=1031123412#gid=1031123412")
nameworksheet = namelist.worksheets()
sheetno = len(nameworksheet)

for i in range(0,sheetno):
  findit(rolltostream , i , nameworksheet[i] , nameworksheet[i].title)

Sheet 3 ➜ 2 13000124121
Sheet 4 ➜ 18 13000123018
Sheet 4 ➜ 52 13000123052
Sheet 5 ➜ 2 13000123067
Sheet 5 ➜ 24 13000123090
Sheet 5 ➜ 28 13000123095
Sheet 5 ➜ 34 13000123101
Sheet 5 ➜ 36 13000123103
Sheet 5 ➜ 38 13000123105
Sheet 5 ➜ 43 13000123110
Sheet 5 ➜ 46 13000123113
Sheet 5 ➜ 50 13000123117
Sheet 5 ➜ 57 13000123124
Sheet 5 ➜ 63 13000124159
Sheet 6 ➜ 45 13000122039


In [74]:
# execute to modify sheet (make sure if the sheet has okay format first)

for i in range(0,sheetno):
  writeit(rolltostream , i , nameworksheet[i] , nameworksheet[i].title)

In [72]:
#check for payment status of all depts

for i in range(0,sheetno):
  check_pay_status(i , nameworksheet[i] , nameworksheet[i].title)

Dept: 1Y - CSE -A  Paid: 0 / 60
Dept: 1Y - CSE - B  Paid: 0 / 60
Dept: 1Y -CSE-C  Paid: 1 / 36
Dept: 2Y - CSE- A  Paid: 2 / 71
Dept: 2Y - CSE- B  Paid: 12 / 70
Dept: 3Y - CSE - A  Paid: 3 / 70
Dept: 3Y - CSE- B  Paid: 0 / 73
Dept: 2Y - CSBS  Paid: 0 / 63
Dept: 3Y - CSBS  Paid: 0 / 65
